In [ ]:
<b> Import Library, find available devices </b>

In [1]:
# Initialization code for 3.3 V SiGe LVR Load Regulation Testing.
# This block identifies and initializes the connected SMUs (Source Measure Units).
# If running for the first time, ensure the SMU names are correctly referenced in the subsequent code block.

# This program sweeps VIN for a fixed VREF and measures total quiescent current (IQ) and VOUT. 
# This program sets up a directory for all measurements taken at a specific temperature by prompting the user for the temperature.
# This program creates a .csv file to save the measurements for a VREF in the temperature directory. 

# The setup assumes the use of two SMUs:
# - One dedicated to VIN sweeps.
# - One Sinks a load current and measures VOUT.

import pyvisa                    # Importing the pyvisa library for instrument communication
import time                      # Importing the time module to handle time-related tasks (e.g., delays)
import matplotlib.pyplot as plt  # Importing matplotlib.pyplot for plotting graphs and visualizing data
import numpy as np               # Importing numpy for numerical operations, particularly with arrays
import pandas as dp              # Importing pandas as dp for data manipulation and analysis
import os                        # Importing os to interact with the operating system, such as handling file paths
import csv                       # Importing csv to read from and write to CSV files
rm = pyvisa.ResourceManager()    # Create a ResourceManager object, which manages communication with instruments
rm.list_resources()              # List all available resources (e.g., connected instruments) managed by pyvisa

('USB0::0x2A8D::0x0101::MY57508183::INSTR',
 'ASRL3::INSTR',
 'ASRL4::INSTR',
 'USB0::0x0957::0x0607::MY45002108::0::INSTR',
 'USB0::0x0957::0x1796::MY57231548::0::INSTR',
 'USB0::0x0957::0x1796::MY57231553::0::INSTR',
 'USB0::0x0957::0x17B6::MY56310504::0::INSTR',
 'USB0::0x0957::0x2C07::MY57890777::0::INSTR',
 'USB0::0x2A8D::0x0101::MY54505624::0::INSTR')

In [2]:
#Establishing Connection for VIN SMU
SM_VIN = rm.open_resource('ASRL3::INSTR')  # Open a connection to the instrument with the address 'ASRL3::INSTR' and assign it to SM_VIN
SM_VIN.read_termination = '\n'             # Set the read termination character to a newline. This tells pyvisa when to consider a message as complete when reading from the instrument.
SM_VIN.write_termination = '\n'            # Set the write termination character to a newline. This ensures that each command sent to the instrument is properly terminated.
SM_VIN.baud_rate = 9600                    # Set the baud rate for serial communication to 9600. This is the speed at which data is transmitted over the serial connection.
print(SM_VIN)                              # Print the resource object (SM_VIN) to display its properties
print(SM_VIN.query('*IDN?'))               # Send the '*IDN?' command to the instrument and print its response. This command usually asks the instrument to identify itself (e.g., manufacturer, model number).

#Establishing Connection for VOUT SMU
SM_VOUT = rm.open_resource('ASRL4::INSTR') # Open a connection to the instrument with the address 'ASRL4::INSTR' and assign it to SM_VOUT
SM_VOUT.read_termination = '\n'            # Set the read termination character to a newline for SM_VOUT, similar to SM_VIN
SM_VOUT.write_termination = '\n'           # Set the write termination character to a newline for SM_VOUT, similar to SM_VIN
SM_VOUT.baud_rate = 9600                   # Set the baud rate for SM_VOUT to 9600, ensuring both instruments communicate at the same speed
print(SM_VOUT)                             # Print the resource object (SM_VOUT) to display its properties
print(SM_VOUT.query('*IDN?'))              # Send the '*IDN?' command to the SM_VOUT instrument and print its response, identifying the instrument

#Establishing Connection for VOUT multimeter
MM_VOUT = rm.open_resource('USB0::10893::257::MY57508183::0::INSTR')
MM_VOUT.read_termination = '\n'             # Set the read termination character to a newline. This tells pyvisa when to consider a message as complete when reading from the instrument.
MM_VOUT.write_termination = '\n'
MM_VOUT.baud_rate = 9600
print(MM_VOUT)                             # Print the resource object (SM_VOUT) to display its properties
print(MM_VOUT.query('*IDN?'))

VIN_MAX=33
VIN_STEP=1
VIN_MIN=9
ILMAX=-.01
VIN=[]
IQ=[]
VOUT=[]

SerialInstrument at ASRL3::INSTR
KEITHLEY INSTRUMENTS INC.,MODEL 2400,1033554,C27   Feb  4 2004 14:58:04/A02  /K/H
SerialInstrument at ASRL4::INSTR
KEITHLEY INSTRUMENTS INC.,MODEL 2400,0904402,C32   Oct  4 2010 14:20:11/A02  /J/H
USBInstrument at USB0::10893::257::MY57508183::0::INSTR
Keysight Technologies,34465A,MY57508183,A.02.17-02.40-02.17-00.52-04-01


In [ ]:
#<b> Initialize Source Meter (VIN) </b>

In [ ]:
SM_VIN.write(":CONF:CURR")                  # Configure the SMU to measure current
print(SM_VIN.query(":CONF?"))               # Query and print the current configuration of the SMU
SM_VIN.write("SENS:CURR:PROT:LEV 0.0105")   # Set the current protection level (compliance) to 10.5 mA. This limits the maximum current to protect the device under test.
SM_VIN.write("SENS:CURR:RANG 1E-2")         # Set the current measurement range to 10 mA. This sets the expected maximum current for accurate measurement.
SM_VIN.write("CURR:NPLC 10")                # Set the number of power line cycles (NPLC) for the current measurement to 10. 
                                            # This controls the integration time, with higher values leading to more accurate but slower measurements.
SM_VIN.write("OUTP:STAT 0")                 # Turn off the VIN source meter

"CURR:DC"


12

In [ ]:
#Initialize Source Meter (Source/Sink)

In [ ]:
SM_VOUT.write(":SOUR:FUNC CURR")             #Configure 
SM_VOUT.write("SOUR:CURR:RANG 0.0105") 
SM_VOUT.write(":CONF:VOLT")                  # Configure the SMU to measure voltage
print(SM_VOUT.query(":CONF?"))               # Query and print the current configuration of the SMU

SM_VOUT.write("SENS:VOLT:PROT:LEV 2.5")      # Set the voltage protection level (compliance) to 2.5 V. This limits the maximum voltage to protect the device under test.
SM_VOUT.write("SENS:VOLT:RANG 2.5")          # Set the voltage measurement range to 10 V. This sets the expected maximum voltage for accurate measurement.
SM_VOUT.write("VOLT:NPLC 10")                # Set the number of power line cycles (NPLC) for the current measurement to 10. 
                                             # This controls the integration time, with higher values leading to more accurate but slower measurements.
SM_VOUT.write("OUTP:STAT 0")                 # Turn off the VIN source meter

-2.985000E-03,-5.366790E-04,+9.910000E+37,+1.150531E+04,+2.150800E+04


12

In [ ]:
#Initialize Multi Meter 

In [5]:
MM_VOUT.write(":CONF:VOLT:DC")
print(MM_VOUT.query(":CONF?"))

"VOLT +1.00000000E+01,+1.00000000E-06"


In [ ]:
#<b> Enter the Device under Test </b>

In [6]:
directory_name = input("Enter the Device under Test (DUT):")  # Prompt the user to input the temperature of the experiment, which will be used as the directory name
DUT="DUT_"+str(directory_name)
os.makedirs(DUT, exist_ok=True)                          # Create a directory with the given name. If the directory already exists, it won't raise an error due to exist_ok=True

In [ ]:
#<b> Enter the Experiment Temperature </b>

In [7]:
sub_directory_name = input("Enter the temperaure of this experiment:")  # Prompt the user to input the temperature of the experiment, which will be used as the directory name
local=str(DUT)+"/TEMP_"+str(sub_directory_name)
os.makedirs(local, exist_ok=True)                          # Create a directory with the given name. If the directory already exists, it won't raise an error due to exist_ok=True

In [ ]:
#<b> VIN Sweep </b>

In [8]:
VREF = input("Enter the reference voltage of this experiment:")  # Prompt the user to input the reference voltage of the experiment, which will be used to name the .csv file.

In [54]:
# Makes list of load currents to test
ILlist=[]
for j in [-.00001, -.0001, -.001]:
    for j2 in range(1,10):
        ILlist.append(j*j2)

SHEET="VREF="+str(VREF)+".csv"                               # Create a CSV file name based on the VREF value
VIN=[]                                                       # Initialize an empty list to store VIN values
IQ=[]                                                        # Initialize an empty list to store IQ (current) values
VOUT=[]                                                      # Initialize an empty list to store VOUT values
VOUTcalc=[]
IL=[]
VOUTSM=[]
RBIASfix = 1500         #TODO: Calculate RBIAS?
ILMAX = ILlist[0]
VL = float(VREF) + RBIASfix * ILMAX                         # Initial estimate for VL, should speed up test TODO: REMOVE?
VLOAD="SOUR:VOLT:LEV:IMM:AMPL " + str(VL)
SM_VOUT.write(VLOAD)                                         # Set the voltage level on the VOUT bias source meter
SM_VIN.write("SOUR:VOLT:LEV:IMM:AMPL 3.3")                  
SM_VIN.write("OUTP:STAT 1")                                  # Turn on the VIN source meter output
time.sleep(.5)
SM_VOUT.write(":SOUR:VOLT:MODE FIX")
SM_VOUT.write("OUTP:STAT 1")                                 # Turn on the VOUT source meter output
SM_VIN.query(":MEAS:CURR?") #TODO: still necessary? could replace with INIT or CONF?
time.sleep(1)

#TODO: Change order of test cases to be the most useful

##################### FIRST: Sweep of IL for VIN = 3.3V #####################
for ILMAX in ILlist:
    # Set the source meter voltage
    VLguess = float(VREF) + RBIASfix * ILMAX                         # Initial estimate for VL, should speed up test
    VL = VLguess
    VLOAD="SOUR:VOLT:LEV:IMM:AMPL " + str(VL)
    SM_VOUT.write(VLOAD)
    time.sleep(.5)          # Wait time before measurement is taken

    # Measure ILOAD and modify VOUT to achieve ILOAD = ILMAX
    VOUTdata = SM_VOUT.query(":MEAS:CURR?").split(',')
    VOUTvolt = MM_VOUT.query(":MEAS:VOLT?")
    if abs(ILMAX) < .0001:
        acc = .05
    else: acc = .01
    while abs(ILMAX - float(VOUTdata[1])) > abs(ILMAX) * acc:
        #TODO: Need to find step resolution 
        #Switch RBIASfix with RBIAS
        step = VLguess/5000
        if float(VOUTdata[1]) > ILMAX:          #sweep down
            start = float(VOUTdata[0])
            step = -step
            stop = start + 20*step
        if float(VOUTdata[1]) < ILMAX:          #sweep up
            start = float(VOUTdata[0])
            step = step
            stop = start + 20*step
        #SM_VOUT.write(":SENS:FUNC:CONC OFF")
        SM_VOUT.write(":SOUR:VOLT:START " + str(start))
        SM_VOUT.write(":SOUR:VOLT:STOP " + str(stop))
        SM_VOUT.write(":SOUR:VOLT:STEP " + str(step))
        SM_VOUT.write(":SOUR:VOLT:MODE SWE")
        count = int(abs((start-stop)/step))
        SM_VOUT.write(":TRIG:COUN " + str(count))
        SM_VOUT.write(":SOUR:DEL 0.1")
        SM_VOUT.write(":INIT")
        time.sleep(count)
        SweepData = SM_VOUT.query(":FETC?").split(',')
        SweepCurrentData =[]
        bestDIFF = 1000
        foundVOLT = False
        for i, ele in enumerate(SweepData):
            if i % 5 == 1:
                SweepCurrentData.append(float(ele))
        for i, ele in enumerate(SweepCurrentData):
            if abs(ILMAX - ele) < bestDIFF:
                bestDIFF = abs(ILMAX - ele)
                bestVOLT = SweepData[i*5]
                if abs(ILMAX - ele) > abs(ILMAX) * acc:
                    bestVOLT = SweepData[5*len(SweepCurrentData) - 5]
        VLOAD="SOUR:VOLT:LEV:IMM:AMPL " + str(bestVOLT)
        SM_VOUT.write(":SOUR:VOLT:MODE FIX")
        SM_VOUT.write(VLOAD)
        time.sleep(1)
        VOUTdata = SM_VOUT.query(":MEAS:CURR?").split(',')
        VOUTvolt = MM_VOUT.query(":MEAS:VOLT?")
        time.sleep(1)
    print(VLOAD)
    VOUT.append(VOUTvolt)
    VOUTcalc.append(VOUTdata[1]*RBIASfix + VOUTdata[0])
    VOUTSM.append(VOUTdata[0])
    IL.append(VOUTdata[1])                                    
    VIN.append(3.3)                                          # Append the current VIN value to the VIN list
    IQ.append(SM_VIN.query(":MEAS:CURR?").split(',')[1])     # Query and append the measured IQ current
    time.sleep(.5)


##################### SECOND: Sweep of VIN for IL=10mA #####################
ILMAX = -.01

VL = float(VREF) + RBIASfix * ILMAX                         # Initial estimate for VL, should speed up test
VLOAD="SOUR:VOLT:LEV:IMM:AMPL " + str(VL)
SM_VOUT.write(VLOAD)

for i in range (VIN_MAX,VIN_MIN,-VIN_STEP):                  # Loop from VIN_MAX to VIN_MIN in steps of VIN_STEP
                                                            # This range will decrement VIN from VIN_MAX to VIN_MIN
    val = i / 10                                             # Convert the current loop index into a floating-point voltage value
    # Set the source meter voltage
    VIN_V = "SOUR:VOLT:LEV:IMM:AMPL " + str(val)             # Create the command string to set VIN voltage
    SM_VIN.write(VIN_V)                                      # Write the command to set the VIN voltage on the source meter
    time.sleep(1)                                            # Pause for 1 second to allow the system to stabilize
    
    # Measure ILOAD and modify VOUT to achieve ILOAD = ILMAX
    VOUTdata = SM_VOUT.query(":MEAS:CURR?").split(',')
    VOUTvolt = MM_VOUT.query(":MEAS:VOLT?")
    acc = .01
    if abs(ILMAX - float(VOUTdata[1])) > abs(ILMAX) * acc and float(VOUTvolt) < float(VREF) - .1:
        SM_VOUT.write("SOUR:VOLT:LEV:IMM:AMPL " + str(float(VOUTdata[0]) - .1))
        time.sleep(1)
        VOUTdata = SM_VOUT.query(":MEAS:CURR?").split(',')
        VOUTvolt = MM_VOUT.query(":MEAS:VOLT?")
    while abs(ILMAX - float(VOUTdata[1])) > abs(ILMAX) * acc:
        RBIAS = (float(VOUTvolt) - VL) / float(VOUTdata[1])
        VL = float(VOUTdata[0]) - (ILMAX - float(VOUTdata[1])) * RBIAS 
        VLOAD="SOUR:VOLT:LEV:IMM:AMPL " + str(VL)
        print(VLOAD)
        SM_VOUT.write(VLOAD)
        time.sleep(1)
        VOUTdata = SM_VOUT.query(":MEAS:CURR?").split(',')
        VOUTvolt = MM_VOUT.query(":MEAS:VOLT?")
    VOUT.append(VOUTvolt)
    VOUTcalc.append(VOUTdata[1]*RBIASfix + VOUTdata[0])
    VOUTSM.append(VOUTdata[0])
    IL.append(VOUTdata[1])                                     #TODO: check if this works, else call query instead of VOUTdata variable
    VIN.append(val)                                          # Append the current VIN value to the VIN list
    IQ.append(SM_VIN.query(":MEAS:CURR?").split(',')[1])     # Query and append the measured IQ current

    # TODO: ADD Check of RBIAS, maybe calculate RBIAS at the beginning of each new temp

# After the loop completes, turn off the source meters
SM_VOUT.write("OUTP:STAT 0")                                 # Turn off the VOUT source meter
SM_VIN.write("OUTP:STAT 0")                                  # Turn off the VIN source meter

# Create a DataFrame with the collected data
df = dp.DataFrame({'VIN': VIN, 'IQ': IQ, 'IL': IL, 'VOUT': VOUT, 'VOUTcalc': VOUTcalc, 'VOUTSM': VOUTSM})      # Create a pandas DataFrame from VIN, IQ, and VOUT lists

csv_path=os.path.join(local, SHEET)                          # Generate the file path for saving the CSV
df.to_csv(csv_path, index=False)                             # Save the DataFrame to a CSV file without including the index

KeyboardInterrupt: 

In [55]:
SM_VOUT.write("OUTP:STAT 0")                                 # Turn off the VOUT source meter
SM_VIN.write("OUTP:STAT 0") 

12